## Scraping Data Analyst Jobs from Indeed

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from IPython.display import HTML
from selenium import webdriver
import time as time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

In [2]:
# Instalation
s = Service((r'C:\Users\eli\Desktop\AllMyRepos\Web-Scraping-with-BeautifulSoup-and-Selenium\Project No.10\chromedriver.exe'))
chromeOptions = Options()
chromeOptions.headless = False
driver = webdriver.Chrome(service=s, options=chromeOptions)

In [3]:
# lists
titles_ = []
companies_ = []
ratings_ = []
locations_ = []
salaries_ = []
links_ = []

In [4]:
url = 'https://www.indeed.com/jobs?q=data+analyst&l=New+York%2C+NY&start=0&pp=gQAPAAABhIvz3qEAAAAB7kC5IwAnAQEBCFFbITFk9V4_HS90YSFZhUXFWI1BeZMmPL72AuEgB78FZWsAAAA&vjk=a44a90be68296825'
while True:
    # Get URL
    driver.get(url)
    time.sleep(3)
    
    # Soup Object
    soup = BeautifulSoup(driver.page_source, 'lxml')
    
    # Results
    results = soup.findAll('div', class_ = 'cardOutline')
    for result in results:
        try:  
            title = result.find('h2', class_ = 'jobTitle').text
            titles_.append(title)
        except:
            titles_.append(np.nan)

        try:
            company = result.find('span', class_ = 'companyName').text
            companies_.append(company)
        except:
            companies_.append(np.nan)

        try:
            rating = result.find('span', class_ = 'ratingNumber').text
            ratings_.append(rating)
        except:
            ratings_.append(np.nan)

        try:
            location = result.find('div', class_ = 'companyLocation').text
            locations_.append(location)
        except:
            locations_.append(location)

        salary = result.find(class_ = 'metadata salary-snippet-container')
        estimated_salary = result.find('div',class_ = 'metadata estimated-salary-container')
        if salary:
            salaries_.append(salary.text)
        elif estimated_salary:
            salaries_.append(estimated_salary.text)
        else:
            salaries_.append(np.nan)
        
        try:
            link = result.find(class_ ='jcs-JobTitle css-jspxzf eu4oa1w0')['href']
            links_.append('https://www.indeed.com' + link)
        except:
            links_.append(np.nan)
    
    next_url1 = soup.find('a', attrs = {'aria-label' : 'Next Page'})
    next_url2 = soup.find('a', attrs = {'aria-label' : 'Next'})
    if next_url1:
        url = 'https://www.indeed.com/' + next_url1.get('href')
    elif next_url2:
        url = 'https://www.indeed.com/' + next_url2.get('href')
    else:
        break

In [5]:
df_job_posts = pd.DataFrame({ 'Title' : titles_,'Company': companies_,'Rating': ratings_, 
                           'Location' : locations_, 'Salary' : salaries_,
                            'Link': links_})

In [6]:
df_job_posts.head(10)

Title  \
0                                       Data Analyst   
1                   Data Analyst, Product Operations   
2       Associate Analyst Reporting & Data Analytics   
3       Data Analyst, CX Data, Forecasting & Systems   
4                                       Data Analyst   
5  Part-time Data Analyst I (ONSITE: 18-19 hours ...   
6                            Part- Time Data Analyst   
7                                       Data Analyst   
8                                       Data Analyst   
9                                       Data Analyst   

                               Company Rating  \
0                      Fox Corporation    4.0   
1                            Riskified    NaN   
2  Madison Square Garden Entertainment    3.9   
3                                 Away    3.0   
4                            Dow Jones    3.9   
5                          Mount Sinai    3.7   
6                        Mercy College    3.9   
7                           S&P Global    3.9   
8                   Roles at YipitData    NaN   
9                 Tradeweb Markets LLC    3.7   

                                     Location  \
0               Hybrid remote in New York, NY   
1         Hybrid remote in New York, NY 10001   
2  New York, NY 10121 (Garment District area)   
3                                New York, NY   
4       New York, NY 10176 (Murray Hill area)   
5                                New York, NY   
6                       Dobbs Ferry, NY 10522   
7                                New York, NY   
8               Hybrid remote in New York, NY   
9           New York, NY 10001 (Chelsea area)   

                             Salary  \
0    Estimated $73.4K - $93K a year   
1         $90,000 - $120,000 a year   
2  Estimated $64.7K - $81.9K a year   
3         $97,000 - $110,000 a year   
4         $40,000 - $160,000 a year   
5          $58,661 - $74,250 a year   
6                       $40 an hour   
7         $74,067 - $128,019 a year   
8   Estimated $83.8K - $106K a year   
9   Estimated $91.9K - $116K a year   

                                                Link  
0  https://www.indeed.com/rc/clk?jk=a58ee3e72a6cb...  
1  https://www.indeed.com/rc/clk?jk=a44a90be68296...  
2  https://www.indeed.com/rc/clk?jk=51e4bd457dbd7...  
3  https://www.indeed.com/rc/clk?jk=5d0e3661f9a3d...  
4  https://www.indeed.com/rc/clk?jk=4823d05ce00f9...  
5  https://www.indeed.com/rc/clk?jk=e954125eddb7b...  
6  https://www.indeed.com/rc/clk?jk=f11df3513be35...  
7  https://www.indeed.com/rc/clk?jk=10edf4b58fccd...  
8  https://www.indeed.com/rc/clk?jk=443bbe1f87bd5...  
9  https://www.indeed.com/rc/clk?jk=73bd6d30f4934...

### Data Cleaning

In [7]:
df_job_posts.drop_duplicates(keep = 'first', inplace = True, ignore_index = False)

In [8]:
df_job_posts['Rating'] = pd.to_numeric(df_job_posts['Rating'], errors = 'coerce')
df_job_posts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 938 entries, 0 to 985
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Title     938 non-null    object 
 1   Company   938 non-null    object 
 2   Rating    744 non-null    float64
 3   Location  938 non-null    object 
 4   Salary    770 non-null    object 
 5   Link      938 non-null    object 
dtypes: float64(1), object(5)
memory usage: 51.3+ KB


In [9]:
HTML(df_job_posts.to_html(render_links = True, escape = False))

In [10]:
df_job_posts.to_excel('data_analyst_job_posts.xlsx', index = False)